# First Run For Best Model

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
from scipy.sparse import csc_matrix, eye, diags
from scipy.sparse.linalg import spsolve
from scipy.optimize import curve_fit
import tensorflow as tf
import keras
from keras import layers, models, regularizers
import pyts
from pyts.transformation.transformation import GADF
import json
import random
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
from sklearn.utils.class_weight import compute_class_weight
import seaborn as sns

# Set random seeds for reproducibility
def set_seed(seed=42):
    np.random.seed(seed)
    tf.random.set_seed(seed)
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)

# Set a fixed seed
SEED = 42
set_seed(SEED)

# Create experiment directory
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
experiment_dir = os.path.join('experiments', f'experiment_{timestamp}')
os.makedirs(experiment_dir, exist_ok=True)
model_dir = os.path.join(experiment_dir, 'models')
results_dir = os.path.join(experiment_dir, 'results')
os.makedirs(model_dir, exist_ok=True)
os.makedirs(results_dir, exist_ok=True)

# Define directories
data_dir = 'data'
synthetic_dir = os.path.join(data_dir, 'synthetic')
maps_dir = os.path.join(data_dir, 'maps')
labels_dir = os.path.join(data_dir, 'labels')
visualizations_dir = os.path.join(data_dir, 'visualizations')
problematic_spectra_dir = os.path.join(data_dir, 'problematic_spectra')
os.makedirs(synthetic_dir, exist_ok=True)
os.makedirs(maps_dir, exist_ok=True)
os.makedirs(labels_dir, exist_ok=True)
os.makedirs(visualizations_dir, exist_ok=True)
os.makedirs(problematic_spectra_dir, exist_ok=True)

# Save experiment configuration
config = {
    'seed': SEED,
    'spectrum_length': 880,
    'image_size': 64,
    'num_synthetic_per_type': 999,
    'num_types': 11,
    'total_spectra': 11000,
    'epochs': 10,
    'batch_size_1d': 64,
    'batch_size_2d': 32,
    'validation_split': 0.1,
    'test_size': 0.2,
    'growth_rate': 12,
    'num_classes': 11,
    'experiment_timestamp': timestamp
}
with open(os.path.join(experiment_dir, 'config.json'), 'w') as f:
    json.dump(config, f, indent=4)

# Baseline functions
def poly_baseline(x, p, intensity, b):
    y = (x / len(x)) ** p + b
    return y * intensity / max(y)

def gaussian_baseline(x, mean, sd, intensity, b):
    y = np.exp(-(x - mean) ** 2 / (2 * sd ** 2)) / (sd * np.sqrt(2 * np.pi)) + b
    return y * intensity / max(y)

def pg_baseline(x, p, in1, mean, sd, in2, b):
    y1 = (x / len(x)) ** p + b
    y2 = np.exp(-(x - mean) ** 2 / (2 * sd ** 2)) / (sd * np.sqrt(2 * np.pi)) + b
    return y1 / max(y1) * in1 + y2 / max(y2) * in2

def mix_min_no(sp, baseline):
    return np.minimum(baseline, sp)

def iterative_fitting_with_bounds_no(sp, model, ite=10):
    fitted_baseline = np.zeros(sp.shape[0])
    x = np.linspace(1, sp.shape[0], sp.shape[0])
    tempb = sp
    torch_tempb = tf.expand_dims(tempb, axis=0)
    i = 0
    while i < ite:
        tadvice = model(torch_tempb)
        if tadvice[0][0] >= 0.5 and tadvice[0][1] >= 0.5:
            try:
                p, c = curve_fit(pg_baseline, x, tempb,
                                bounds=([1, 0.5, 0, 100, 0.5, -0.5], [3, 1, sp.shape[0], 600, 1, 0.5]),
                                maxfev=10000)
                fitted_baseline = pg_baseline(x, p[0], p[1], p[2], p[3], p[4], p[5])
            except RuntimeError:
                fitted_baseline = tempb
        elif tadvice[0][0] >= 0.5:
            try:
                p, c = curve_fit(poly_baseline, x, tempb,
                                bounds=([1, 0.5, -0.5], [3, 1, 0.5]),
                                maxfev=10000)
                fitted_baseline = poly_baseline(x, p[0], p[1], p[2])
            except RuntimeError:
                fitted_baseline = tempb
        elif tadvice[0][1] >= 0.5:
            try:
                p, c = curve_fit(gaussian_baseline, x, tempb,
                                bounds=([0, 100, 0.5, -0.5], [sp.shape[0], 600, 1, 0.5]),
                                maxfev=10000)
                fitted_baseline = gaussian_baseline(x, p[0], p[1], p[2], p[3])
            except RuntimeError:
                fitted_baseline = tempb
        tempb = mix_min_no(tempb, fitted_baseline)
        tempb_np = np.array(tempb)
        torch_tempb = tempb_np.reshape(1, sp.shape[0])
        i += 1
    return tempb

def create_baseline_model(input_shape=880):
    model = models.Sequential([
        layers.Input(shape=(input_shape,)),
        layers.Reshape((input_shape, 1)),
        layers.Conv1D(filters=16, kernel_size=5, strides=1, activation='relu'),
        layers.AveragePooling1D(pool_size=2, strides=2),
        layers.Flatten(),
        layers.Dense(100, activation='relu'),
        layers.Dense(2, activation='sigmoid')
    ])
    return model

def train_baseline_model(baseline_model, noise_data, epochs=10, batch_size=32):
    # Load labels from labels_noise_pure_182.npy
    try:
        labels = np.load(os.path.join(data_dir, 'labels_noise_pure_182.npy'))
        print("Đã tải nhãn từ labels_noise_pure_182.npy thành công!")
    except Exception as e:
        print(f"Lỗi khi tải nhãn: {e}. Sử dụng nhãn ngẫu nhiên.")
        labels = np.random.randint(0, 2, size=noise_data.shape[0])

    X = []
    y = []
    for i in range(noise_data.shape[0]):
        pure = noise_data[i, 0, 0, :, 0]
        noisy = noise_data[i, 0, 1, :, 0]
        X.append(noisy)
        y.append(labels[i])  # Sử dụng nhãn thực
    X = np.array(X)[:, :, np.newaxis]
    y = np.array(y)
    baseline_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    baseline_model.fit(X, y, epochs=epochs, batch_size=batch_size, validation_split=0.1)
    baseline_model.save_weights(os.path.join(data_dir, 'model.weights.h5'))
    return baseline_model

# Normalize spectrum
def normalize_spectrum(spectrum):
    spectrum = spectrum - np.min(spectrum)
    if np.max(spectrum) > 0:
        spectrum = spectrum / np.max(spectrum)
    return spectrum

# WhittakerSmooth and airPLS
def WhittakerSmooth(x, w, lambda_=1, differences=1):
    X = np.matrix(x)
    m = X.size
    E = eye(m, format='csc')
    for i in range(differences):
        E = E[1:] - E[:-1]
    W = diags(w, 0, shape=(m, m))
    A = csc_matrix(W + (lambda_ * E.T * E))
    B = csc_matrix(W * X.T)
    background = spsolve(A, B)
    return np.array(background)

def airPLS(x, lambda_=100, porder=1, itermax=15):
    m = x.shape[0]
    w = np.ones(m)
    lambda_ = max(50, min(500, 50 * np.std(x) / (np.mean(np.abs(x)) + 1e-6)))
    for i in range(1, itermax + 1):
        z = WhittakerSmooth(x, w, lambda_, porder)
        d = x - z
        dssn = np.abs(d[d < 0].sum())
        if dssn < 0.001 * np.abs(x).sum():
            return z
        if i == itermax:
            print(f'WARNING: Max iteration reached! lambda_={lambda_:.2f}, dssn={dssn:.2e}')
            np.save(os.path.join(problematic_spectra_dir, f'problematic_spectrum_{np.random.randint(1000000)}.npy'), x)
            return WhittakerSmooth(x, np.ones(m), lambda_=50)
        w[d >= 0] = 0
        w[d < 0] = np.exp(i * np.abs(d[d < 0]) / dssn)
        w[0] = np.exp(i * (d[d < 0]).max() / dssn)
        w[-1] = w[0]
    return z

# Enhanced baseline correction
def enhanced_baseline_correction(spectrum, baseline_model):
    """
    Trừ nền: Dùng baseline_model để đoán và trừ nền sơ bộ (đa thức/Gaussian),
    sau đó dùng airPLS để tinh chỉnh. Nếu lỗi, dùng airPLS trực tiếp.
    """
    try:
        baseline = iterative_fitting_with_bounds_no(spectrum, baseline_model)
        fine_corrected = airPLS(baseline, lambda_=100, itermax=15)
        return np.clip(spectrum - fine_corrected, 0, None)
    except Exception as e:
        print(f"Error in baseline correction: {e}. Falling back to airPLS.")
        return np.clip(spectrum - airPLS(spectrum, lambda_=100, itermax=15), 0, None)

# Interpolate spectrum
def interpolate_spectrum(spectrum, original_length, target_length=880):
    x_original = np.linspace(0, original_length - 1, original_length)
    x_target = np.linspace(0, original_length - 1, target_length)
    interpolator = interp1d(x_original, spectrum, kind='linear', fill_value="extrapolate")
    return interpolator(x_target)

# Shift spectrum
def shift_spectrum(spectrum, shift):
    return np.roll(spectrum, shift)

# Stretch spectrum
def stretch_spectrum(spectrum, alpha):
    original_len = len(spectrum)
    new_len = int(original_len / alpha)
    if new_len < 1:
        new_len = 1
    if new_len > original_len * 10:
        new_len = original_len * 10
    x_original = np.linspace(0, original_len - 1, original_len)
    x_new = np.linspace(0, original_len - 1, new_len)
    interpolator = interp1d(x_original, spectrum, kind='linear', fill_value="extrapolate")
    stretched = interpolator(x_new)
    return interpolate_spectrum(stretched, new_len, original_len)

# Generate synthetic spectrum
def generate_synthetic_spectrum(input_spectrum, noise_data, spectrum_length=880):
    """
    Tạo phổ tổng hợp từ một phổ gốc duy nhất:
    - Thêm nhiễu từ noise_data và nhiễu Gaussian.
    - Thêm đường nền (đa thức/Gaussian hoặc không).
    - Dịch chuyển hoặc kéo dãn/nén.
    - Không trộn với phổ khác.
    """
    x_range = np.linspace(0, spectrum_length, spectrum_length)
    # Add noise from dataset
    noise_idx = np.random.randint(0, noise_data.shape[0])
    pure = noise_data[noise_idx, 0, 0, :, 0]
    noisy = noise_data[noise_idx, 0, 1, :, 0]
    noise = noisy - pure  # Shape (880,)
    scale = np.random.uniform(1.0, 2.0)
    synthetic_spectrum = input_spectrum + noise * scale
    # Add Gaussian noise
    synthetic_spectrum += np.random.normal(0, 0.05 * np.std(input_spectrum), spectrum_length)
    # Add synthetic baseline
    baseline_type = np.random.choice(['poly', 'gaussian', 'none'], p=[0.3, 0.3, 0.4])
    if baseline_type == 'poly':
        baseline = poly_baseline(x_range, p=np.random.uniform(1.9, 2.1),
                                intensity=np.random.uniform(0.75, 0.8),
                                b=np.random.uniform(-0.1, 0.1))
        synthetic_spectrum += baseline
    elif baseline_type == 'gaussian':
        baseline = gaussian_baseline(x_range, mean=np.random.uniform(0, spectrum_length),
                                   sd=np.random.uniform(250, 300),
                                   intensity=np.random.uniform(0.75, 0.8),
                                   b=np.random.uniform(-0.1, 0.1))
        synthetic_spectrum += baseline
    # Probabilistic augmentation
    aug_type = np.random.choice(['none', 'shift', 'stretch'], p=[0.5, 0.25, 0.25])
    if aug_type == 'shift':
        shift = np.random.randint(-10, 11)
        synthetic_spectrum = shift_spectrum(synthetic_spectrum, shift)
    elif aug_type == 'stretch':
        alpha = np.random.uniform(0.5, 2.0)
        synthetic_spectrum = stretch_spectrum(synthetic_spectrum, alpha)
    return synthetic_spectrum

# Create GADF map
def create_gadf_map(spectrum, image_size=64):
    spectrum = normalize_spectrum(spectrum)
    spectrum = 2 * spectrum - 1
    target_length = image_size * (spectrum.shape[0] // image_size)
    if target_length != spectrum.shape[0]:
        spectrum = interpolate_spectrum(spectrum, spectrum.shape[0], target_length)
    gadf = GADF(image_size=image_size, overlapping=False, scale='-1')
    return gadf.fit_transform(spectrum.reshape(1, -1))[0][:, :, np.newaxis]

# Load Excel data
def load_excel_data():
    excel_path = os.path.join(data_dir, 'Ethanol_Methanol.xlsx')
    try:
        df = pd.read_excel(excel_path, usecols='A:L')
        raman_shift = df['Raman Shift (cm-1)'].values
        spectra_data = {
            'ethanol': df['Ethanol'].values,
            'methanol': df['Methanol'].values,
            'EM1_a': df['EM1_a'].values,
            'EM2_a': df['EM2_a'].values,
            'EM3_a': df['EM3_a'].values,
            'EM4_a': df['EM4_a'].values,
            'EM5_a': df['EM5_a'].values,
            'EM6_a': df['EM6_a'].values,
            'EM7_a': df['EM7_a'].values,
            'EM8_a': df['EM8_a'].values,
            'EM9_a': df['EM9_a'].values
        }
        for key in spectra_data:
            spectrum = spectra_data[key]
            spectrum = spectrum[~np.isnan(spectrum)]
            spectra_data[key] = interpolate_spectrum(spectrum, len(spectrum), 880)
        return spectra_data, raman_shift
    except Exception as e:
        print(f"Lỗi khi tải dữ liệu từ {excel_path}: {e}")
        return {}, np.array([])

# Load noise data
def load_noise_data():
    try:
        # Noise data shape: (15000, 1, 2, 880, 1) - 15000 samples, 2 spectra (pure, noisy), 880 points
        noise_data = np.load(os.path.join(data_dir, 'dataset_noise_pure_182.npy'))
        return noise_data
    except Exception as e:
        print(f"Lỗi khi tải dữ liệu nhiễu: {e}")
        return np.array([])

# Process data
spectra_data, raman_shift = load_excel_data()
noise_data = load_noise_data()
if not spectra_data or noise_data.size == 0:
    raise FileNotFoundError("Không thể tải dữ liệu từ file Excel hoặc file nhiễu.")

# Initialize and train baseline model
baseline_model = create_baseline_model(input_shape=880)
try:
    baseline_model.load_weights(os.path.join(data_dir, 'model.weights.h5'))
    print("Trọng số mô hình baseline đã được tải thành công!")
except Exception as e:
    print(f"Lỗi khi tải trọng số: {e}. Training baseline model.")
    baseline_model = train_baseline_model(baseline_model, noise_data)

# Generate synthetic data
X_1d = []
X_2d = []
labels = []
sample_ids = []
example_spectra = {}
total_spectra = 0
spectrum_length = 880

ratio_to_label = {0.0: 0, 0.1: 1, 0.2: 2, 0.3: 3, 0.4: 4, 0.5: 5, 0.6: 6, 0.7: 7, 0.8: 8, 0.9: 9, 1.0: 10}
ratios = {
    'ethanol': 1.0,
    'methanol': 0.0,
    'EM1_a': 0.9,
    'EM2_a': 0.8,
    'EM3_a': 0.7,
    'EM4_a': 0.6,
    'EM5_a': 0.5,
    'EM6_a': 0.4,
    'EM7_a': 0.3,
    'EM8_a': 0.2,
    'EM9_a': 0.1
}

print("Bắt đầu sinh dữ liệu tổng hợp...")
for spectrum_type, ethanol_ratio in ratios.items():
    print(f"Xử lý {spectrum_type} với ethanol ratio {ethanol_ratio}...")
    input_spectrum = spectra_data[spectrum_type]
    normalized_raw = normalize_spectrum(input_spectrum)
    corrected_spectrum = enhanced_baseline_correction(normalized_raw, baseline_model)
    corrected_spectrum = normalize_spectrum(corrected_spectrum)
    X_1d.append(corrected_spectrum)
    X_2d.append(create_gadf_map(corrected_spectrum, image_size=64))
    label_idx = ratio_to_label[ethanol_ratio]
    labels.append(label_idx)
    sample_ids.append(f"{spectrum_type}_original")
    total_spectra += 1
    if total_spectra == 1 or ethanol_ratio not in example_spectra:
        example_spectra[ethanol_ratio] = {
            'raw': normalized_raw,
            'corrected': corrected_spectrum,
            'ethanol': spectra_data['ethanol'] if ethanol_ratio > 0 else None,
            'methanol': spectra_data['methanol'] if ethanol_ratio < 1 else None
        }
    for i in range(999):
        synthetic_spectrum = generate_synthetic_spectrum(normalized_raw, noise_data, spectrum_length)
        corrected_spectrum = enhanced_baseline_correction(synthetic_spectrum, baseline_model)
        corrected_spectrum = normalize_spectrum(corrected_spectrum)
        X_1d.append(corrected_spectrum)
        X_2d.append(create_gadf_map(corrected_spectrum, image_size=64))
        labels.append(label_idx)
        sample_ids.append(f"{spectrum_type}_synthetic_{i}")
        total_spectra += 1
        if i == 0:
            example_spectra[ethanol_ratio] = {
                'raw': synthetic_spectrum,
                'corrected': corrected_spectrum,
                'ethanol': spectra_data['ethanol'] if ethanol_ratio > 0 else None,
                'methanol': spectra_data['methanol'] if ethanol_ratio < 1 else None
            }
        if total_spectra % 1000 == 0:
            print(f"Đã xử lý {total_spectra} phổ.")

print(f"Tổng số phổ: {total_spectra}")

# Convert to numpy arrays
X_1d = np.array(X_1d)[:, :, np.newaxis]
X_2d = np.array(X_2d)
labels_df = pd.DataFrame({'label': labels, 'sample_id': sample_ids})

print("X_1d shape:", X_1d.shape)
print("X_2d shape:", X_2d.shape)
print("labels_df shape:", labels_df.shape)
print("Label distribution:\n", labels_df["label"].value_counts())

# Save data
labels_df.to_csv(os.path.join(labels_dir, "labels.csv"), index=False)
np.save(os.path.join(synthetic_dir, "synthetic_1d.npy"), X_1d)
np.save(os.path.join(maps_dir, "spectral_maps_gadf.npy"), X_2d)

# Plot spectra
def plot_spectra_comparison(wavenumbers, raw_spectrum, corrected_spectrum, ethanol_spectrum, methanol_spectrum, title, filename):
    plt.figure(figsize=(15, 10))
    plt.subplot(2, 1, 1)
    plt.plot(wavenumbers, raw_spectrum, label="Raw Spectrum", color='blue')
    plt.axvspan(1000, 1020, color='red', alpha=0.2, label="Vùng Methanol")
    plt.axvspan(870, 890, color='green', alpha=0.2, label="Vùng Ethanol")
    plt.xlabel("Bước sóng (cm⁻¹)")
    plt.ylabel("Cường độ chuẩn hóa")
    plt.title(f"Phổ Raw ({title})")
    plt.grid(True)
    plt.legend()
    plt.subplot(2, 1, 2)
    plt.plot(wavenumbers, corrected_spectrum, label="Corrected Spectrum", color='orange')
    if ethanol_spectrum is not None:
        plt.plot(wavenumbers, normalize_spectrum(ethanol_spectrum), label="Ethanol Component", color='green', linestyle='--')
    if methanol_spectrum is not None:
        plt.plot(wavenumbers, normalize_spectrum(methanol_spectrum), label="Methanol Component", color='red', linestyle='--')
    plt.axvspan(1000, 1020, color='red', alpha=0.2)
    plt.axvspan(870, 890, color='green', alpha=0.2)
    plt.xlabel("Bước sóng (cm⁻¹)")
    plt.ylabel("Cường độ chuẩn hóa")
    plt.title(f"Phổ Sau Trừ Nền và Thành Phần ({title})")
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.savefig(os.path.join(visualizations_dir, filename), dpi=300, bbox_inches='tight')
    plt.close()

# Sử dụng phạm vi wavenumbers từ 500–3500 cm⁻¹
wavenumbers = np.linspace(500, 3500, 880)
for ratio in example_spectra:
    title = f"Ratio Ethanol/Methanol = {ratio:.2f}/{1-ratio:.2f}" if ratio < 1.0 else "Pure Ethanol" if ratio == 1.0 else "Pure Methanol"
    plot_spectra_comparison(
        wavenumbers,
        example_spectra[ratio]['raw'],
        example_spectra[ratio]['corrected'],
        example_spectra[ratio]['ethanol'],
        example_spectra[ratio]['methanol'],
        title,
        f"spectra_comparison_ratio_{ratio:.2f}.png"
    )

# Define DenseNet models
def build_1d_densenet(input_shape=(880, 1), num_classes=11, growth_rate=12):
    inputs = layers.Input(shape=input_shape)
    x = layers.Conv1D(48, 7, padding='same', activation='relu', kernel_regularizer=regularizers.l2(0.0005))(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling1D(pool_size=2)(x)
    def dense_block(x, num_layers, filters):
        for _ in range(num_layers):
            y = layers.BatchNormalization()(x)
            y = layers.Activation('relu')(y)
            y = layers.Conv1D(filters, 3, padding='same', kernel_regularizer=regularizers.l2(0.0005))(y)
            x = layers.Concatenate()([x, y])
        return x
    def transition_layer(x):
        filters = x.shape[-1]
        x = layers.BatchNormalization()(x)
        x = layers.Activation('relu')(x)
        x = layers.Conv1D(filters // 2, 1, padding='same', kernel_regularizer=regularizers.l2(0.0005))(x)
        x = layers.MaxPooling1D(pool_size=2)(x)
        return x
    for _ in range(3):
        x = dense_block(x, num_layers=4, filters=growth_rate)
        x = transition_layer(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dropout(0.4)(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)
    return models.Model(inputs, outputs)

def build_2d_densenet(input_shape=(64, 64, 1), num_classes=11, growth_rate=12):
    inputs = layers.Input(shape=input_shape)
    x = layers.Conv2D(48, 3, padding='same', activation='relu', kernel_regularizer=regularizers.l2(0.0005))(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)
    def dense_block(x, num_layers, filters):
        for _ in range(num_layers):
            y = layers.BatchNormalization()(x)
            y = layers.Activation('relu')(y)
            y = layers.Conv2D(filters, 3, padding='same', kernel_regularizer=regularizers.l2(0.0005))(y)
            x = layers.Concatenate()([x, y])
        return x
    def transition_layer(x):
        filters = x.shape[-1]
        x = layers.BatchNormalization()(x)
        x = layers.Activation('relu')(x)
        x = layers.Conv2D(filters // 2, 1, padding='same', kernel_regularizer=regularizers.l2(0.0005))(x)
        x = layers.MaxPooling2D(pool_size=(2, 2))(x)
        return x
    for _ in range(3):
        x = dense_block(x, num_layers=4, filters=growth_rate)
        x = transition_layer(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dropout(0.4)(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)
    return models.Model(inputs, outputs)

# Define ResNet models
def build_1d_resnet(input_shape=(880, 1), num_classes=11):
    inputs = layers.Input(shape=input_shape)
    x = layers.Conv1D(64, 5, padding='same', activation='relu', kernel_regularizer=regularizers.l2(0.0001))(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling1D(pool_size=2)(x)
    def residual_block(x, filters, kernel_size=3):
        shortcut = x
        x = layers.Conv1D(filters, kernel_size, padding='same', activation='relu', kernel_regularizer=regularizers.l2(0.0001))(x)
        x = layers.BatchNormalization()(x)
        x = layers.Conv1D(filters, kernel_size, padding='same', activation='relu', kernel_regularizer=regularizers.l2(0.0001))(x)
        x = layers.BatchNormalization()(x)
        if shortcut.shape[-1] != filters:
            shortcut = layers.Conv1D(filters, 1, padding='same')(shortcut)
        x = layers.Add()([shortcut, x])
        x = layers.Activation('relu')(x)
        return x
    x = residual_block(x, 64)
    x = residual_block(x, 64)
    x = layers.MaxPooling1D(pool_size=2)(x)
    x = residual_block(x, 128)
    x = residual_block(x, 128)
    x = residual_block(x, 128)
    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dropout(0.3)(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)
    return models.Model(inputs, outputs)

def build_2d_resnet(input_shape=(64, 64, 1), num_classes=11):
    inputs = layers.Input(shape=input_shape)
    x = layers.Conv2D(32, 3, padding='same', activation='relu', kernel_regularizer=regularizers.l2(0.0005))(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)
    def residual_block(x, filters, kernel_size=3):
        shortcut = x
        x = layers.Conv2D(filters, kernel_size, padding='same', activation='relu', kernel_regularizer=regularizers.l2(0.0005))(x)
        x = layers.BatchNormalization()(x)
        x = layers.Conv2D(filters, kernel_size, padding='same')(x)
        x = layers.BatchNormalization()(x)
        if shortcut.shape[-1] != filters:
            shortcut = layers.Conv2D(filters, 1, padding='same')(shortcut)
        x = layers.Add()([shortcut, x])
        x = layers.Activation('relu')(x)
        return x
    x = residual_block(x, 32)
    x = residual_block(x, 32)
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)
    return models.Model(inputs, outputs)

# Plot confusion matrix
def plot_confusion_matrix(y_true, y_pred, title, filename):
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False,
                xticklabels=[f"{i*10}% Ethanol" for i in range(11)],
                yticklabels=[f"{i*10}% Ethanol" for i in range(11)])
    plt.title(title)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.savefig(os.path.join(visualizations_dir, filename), dpi=300, bbox_inches='tight')
    plt.close()

# Data augmentation
data_augmentation_1d = models.Sequential([
    layers.Lambda(lambda x: x + tf.random.normal(tf.shape(x), mean=0.0, stddev=0.05)),
    layers.Lambda(lambda x: x * tf.random.uniform((), 0.8, 1.2)),
    layers.Lambda(lambda x: tf.roll(x, shift=tf.random.uniform((), -5, 5, dtype=tf.int32), axis=1))
])
data_augmentation_2d = models.Sequential([
    layers.Lambda(lambda x: x + tf.random.normal(tf.shape(x), mean=0.0, stddev=0.05)),
    layers.Lambda(lambda x: x * tf.random.uniform((), 0.8, 1.2)),
    layers.Lambda(lambda x: tf.roll(x, shift=tf.random.uniform((), -5, 5, dtype=tf.int32), axis=1))
])

# Split data
y = labels_df["label"].values
X_1d_train, X_1d_test, y_train, y_test = train_test_split(X_1d, y, test_size=0.2, random_state=42)
X_2d_train, X_2d_test, y_train_2d, y_test_2d = train_test_split(X_2d, y, test_size=0.2, random_state=42)

# Compute class weights
class_weights = compute_class_weight('balanced', classes=np.arange(11), y=y)
class_weight = {i: w for i, w in enumerate(class_weights)}

# Train models with fresh optimizers
early_stopping = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)

# DenseNet 1D
tf.keras.backend.clear_session()
lr_schedule_1d_densenet = keras.optimizers.schedules.CosineDecay(initial_learning_rate=0.001, decay_steps=10*len(X_1d_train)//64)
optimizer_1d_densenet = keras.optimizers.Adam(learning_rate=lr_schedule_1d_densenet)
densenet_1d = models.Sequential([data_augmentation_1d, build_1d_densenet()])
densenet_1d.compile(optimizer=optimizer_1d_densenet, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
densenet_1d.fit(X_1d_train, y_train, validation_split=0.1, epochs=10, batch_size=64,
                callbacks=[keras.callbacks.ModelCheckpoint(os.path.join(model_dir, "best_densenet_1d.keras"), save_best_only=True), early_stopping],
                class_weight=class_weight)

# DenseNet 2D
tf.keras.backend.clear_session()
lr_schedule_2d_densenet = keras.optimizers.schedules.CosineDecay(initial_learning_rate=0.001, decay_steps=10*len(X_2d_train)//32)
optimizer_2d_densenet = keras.optimizers.Adam(learning_rate=lr_schedule_2d_densenet)
densenet_2d = models.Sequential([data_augmentation_2d, build_2d_densenet()])
densenet_2d.compile(optimizer=optimizer_2d_densenet, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
densenet_2d.fit(X_2d_train, y_train_2d, validation_split=0.1, epochs=10, batch_size=32,
                callbacks=[keras.callbacks.ModelCheckpoint(os.path.join(model_dir, "best_densenet_2d.keras"), save_best_only=True), early_stopping],
                class_weight=class_weight)

# ResNet 1D
tf.keras.backend.clear_session()
lr_schedule_1d_resnet = keras.optimizers.schedules.CosineDecay(initial_learning_rate=0.001, decay_steps=10*len(X_1d_train)//64)
optimizer_1d_resnet = keras.optimizers.Adam(learning_rate=lr_schedule_1d_resnet)
resnet_1d = models.Sequential([data_augmentation_1d, build_1d_resnet()])
resnet_1d.compile(optimizer=optimizer_1d_resnet, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
resnet_1d.fit(X_1d_train, y_train, validation_split=0.1, epochs=10, batch_size=64,
              callbacks=[keras.callbacks.ModelCheckpoint(os.path.join(model_dir, "best_resnet_1d.keras"), save_best_only=True), early_stopping],
              class_weight=class_weight)

# ResNet 2D
tf.keras.backend.clear_session()
lr_schedule_2d_resnet = keras.optimizers.schedules.CosineDecay(initial_learning_rate=0.001, decay_steps=10*len(X_2d_train)//32)
optimizer_2d_resnet = keras.optimizers.Adam(learning_rate=lr_schedule_2d_resnet)
resnet_2d = models.Sequential([data_augmentation_2d, build_2d_resnet()])
resnet_2d.compile(optimizer=optimizer_2d_resnet, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
resnet_2d.fit(X_2d_train, y_train_2d, validation_split=0.1, epochs=10, batch_size=32,
              callbacks=[keras.callbacks.ModelCheckpoint(os.path.join(model_dir, "best_resnet_2d.keras"), save_best_only=True), early_stopping],
              class_weight=class_weight)

# Evaluate
y_pred_1d_densenet = densenet_1d.predict(X_1d_test)
y_pred_2d_densenet = densenet_2d.predict(X_2d_test)
y_pred_1d_resnet = resnet_1d.predict(X_1d_test)
y_pred_2d_resnet = resnet_2d.predict(X_2d_test)

y_pred_1d_densenet_labels = np.argmax(y_pred_1d_densenet, axis=1)
y_pred_2d_densenet_labels = np.argmax(y_pred_2d_densenet, axis=1)
y_pred_1d_resnet_labels = np.argmax(y_pred_1d_resnet, axis=1)
y_pred_2d_resnet_labels = np.argmax(y_pred_2d_resnet, axis=1)

densenet_1d_acc = np.mean(y_pred_1d_densenet_labels == y_test)
densenet_2d_acc = np.mean(y_pred_2d_densenet_labels == y_test_2d)
resnet_1d_acc = np.mean(y_pred_1d_resnet_labels == y_test)
resnet_2d_acc = np.mean(y_pred_2d_resnet_labels == y_test_2d)

densenet_1d_precision = precision_score(y_test, y_pred_1d_densenet_labels, average='macro')
densenet_2d_precision = precision_score(y_test_2d, y_pred_2d_densenet_labels, average='macro')
resnet_1d_precision = precision_score(y_test, y_pred_1d_resnet_labels, average='macro')
resnet_2d_precision = precision_score(y_test_2d, y_pred_2d_resnet_labels, average='macro')

densenet_1d_recall = recall_score(y_test, y_pred_1d_densenet_labels, average='macro')
densenet_2d_recall = recall_score(y_test_2d, y_pred_2d_densenet_labels, average='macro')
resnet_1d_recall = recall_score(y_test, y_pred_1d_resnet_labels, average='macro')
resnet_2d_recall = recall_score(y_test_2d, y_pred_2d_resnet_labels, average='macro')

densenet_1d_f1 = f1_score(y_test, y_pred_1d_densenet_labels, average='macro')
densenet_2d_f1 = f1_score(y_test_2d, y_pred_2d_densenet_labels, average='macro')
resnet_1d_f1 = f1_score(y_test, y_pred_1d_resnet_labels, average='macro')
resnet_2d_f1 = f1_score(y_test_2d, y_pred_2d_resnet_labels, average='macro')

print("\nKết quả đánh giá:")
print(f"DenseNet 1D - Accuracy: {densenet_1d_acc:.4f}, Precision: {densenet_1d_precision:.4f}, Recall: {densenet_1d_recall:.4f}, F1: {densenet_1d_f1:.4f}")
print(f"DenseNet 2D (GADF) - Accuracy: {densenet_2d_acc:.4f}, Precision: {densenet_2d_precision:.4f}, Recall: {densenet_2d_recall:.4f}, F1: {densenet_2d_f1:.4f}")
print(f"ResNet 1D - Accuracy: {resnet_1d_acc:.4f}, Precision: {resnet_1d_precision:.4f}, Recall: {resnet_1d_recall:.4f}, F1: {resnet_1d_f1:.4f}")
print(f"ResNet 2D (GADF) - Accuracy: {resnet_2d_acc:.4f}, Precision: {resnet_2d_precision:.4f}, Recall: {resnet_2d_recall:.4f}, F1: {resnet_2d_f1:.4f}")

# Save results
plot_confusion_matrix(y_test, y_pred_1d_densenet_labels, "Confusion Matrix - DenseNet 1D", "cm_densenet_1d.png")
plot_confusion_matrix(y_test_2d, y_pred_2d_densenet_labels, "Confusion Matrix - DenseNet 2D (GADF)", "cm_densenet_2d_gadf.png")
plot_confusion_matrix(y_test, y_pred_1d_resnet_labels, "Confusion Matrix - ResNet 1D", "cm_resnet_1d.png")
plot_confusion_matrix(y_test_2d, y_pred_2d_resnet_labels, "Confusion Matrix - ResNet 2D (GADF)", "cm_resnet_2d_gadf.png")

densenet_1d.save(os.path.join(model_dir, 'densenet_1d_full.keras'))
densenet_2d.save(os.path.join(model_dir, 'densenet_2d_full.keras'))
resnet_1d.save(os.path.join(model_dir, 'resnet_1d_full.keras'))
resnet_2d.save(os.path.join(model_dir, 'resnet_2d_full.keras'))

np.save(os.path.join(results_dir, 'y_pred_1d_densenet.npy'), y_pred_1d_densenet)
np.save(os.path.join(results_dir, 'y_pred_2d_densenet.npy'), y_pred_2d_densenet)
np.save(os.path.join(results_dir, 'y_pred_1d_resnet.npy'), y_pred_1d_resnet)
np.save(os.path.join(results_dir, 'y_pred_2d_resnet.npy'), y_pred_2d_resnet)

metrics = {
    'densenet_1d': {'accuracy': densenet_1d_acc, 'precision': densenet_1d_precision, 'recall': densenet_1d_recall, 'f1': densenet_1d_f1},
    'densenet_2d': {'accuracy': densenet_2d_acc, 'precision': densenet_2d_precision, 'recall': densenet_2d_recall, 'f1': densenet_2d_f1},
    'resnet_1d': {'accuracy': resnet_1d_acc, 'precision': resnet_1d_precision, 'recall': resnet_1d_recall, 'f1': resnet_1d_f1},
    'resnet_2d': {'accuracy': resnet_2d_acc, 'precision': resnet_2d_precision, 'recall': resnet_2d_recall, 'f1': resnet_2d_f1}
}
with open(os.path.join(results_dir, 'metrics.json'), 'w') as f:
    json.dump(metrics, f, indent=4)

KeyboardInterrupt: 

# Re-run Model (best-model form experiment)

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import keras
from keras import layers, models, regularizers
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
from sklearn.utils.class_weight import compute_class_weight
import seaborn as sns
import json

# Set random seeds for reproducibility
def set_seed(seed=42):
    np.random.seed(seed)
    tf.random.set_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)

# Set a fixed seed
SEED = 42
set_seed(SEED)

# Define directories
data_dir = 'data'
synthetic_dir = os.path.join(data_dir, 'synthetic')
maps_dir = os.path.join(data_dir, 'maps')
labels_dir = os.path.join(data_dir, 'labels')
visualizations_dir = os.path.join(data_dir, 'visualizations')
experiment_dir = os.path.join('experiments', 'experiment_20250914_220130')  # Thay bằng timestamp của lần chạy trước
model_dir = os.path.join(experiment_dir, 'models')
results_dir = os.path.join(experiment_dir, 'results')
os.makedirs(model_dir, exist_ok=True)
os.makedirs(results_dir, exist_ok=True)

# Load saved data
X_1d = np.load(os.path.join(synthetic_dir, "synthetic_1d.npy"))
X_2d = np.load(os.path.join(maps_dir, "spectral_maps_gadf.npy"))
labels_df = pd.read_csv(os.path.join(labels_dir, "labels.csv"))
y = labels_df["label"].values

print("X_1d shape:", X_1d.shape)
print("X_2d shape:", X_2d.shape)
print("labels_df shape:", labels_df.shape)
print("Label distribution:\n", labels_df["label"].value_counts())

# Define baseline model (for completeness, in case needed)
def create_baseline_model(input_shape=880):
    model = models.Sequential([
        layers.Input(shape=(input_shape,)),
        layers.Reshape((input_shape, 1)),
        layers.Conv1D(filters=16, kernel_size=5, strides=1, activation='relu'),
        layers.AveragePooling1D(pool_size=2, strides=2),
        layers.Flatten(),
        layers.Dense(100, activation='relu'),
        layers.Dense(2, activation='sigmoid')
    ])
    return model

def train_baseline_model(baseline_model, noise_data, epochs=10, batch_size=32):
    try:
        labels = np.load(os.path.join(data_dir, 'labels_noise_pure_182.npy'))
        print("Đã tải nhãn từ labels_noise_pure_182.npy thành công!")
    except Exception as e:
        print(f"Lỗi khi tải nhãn: {e}. Sử dụng nhãn ngẫu nhiên.")
        labels = np.random.randint(0, 2, size=noise_data.shape[0])

    X = []
    y = []
    for i in range(noise_data.shape[0]):
        pure = noise_data[i, 0, 0, :, 0]
        noisy = noise_data[i, 0, 1, :, 0]
        X.append(noisy)
        y.append(labels[i])
    X = np.array(X)[:, :, np.newaxis]
    y = np.array(y)
    baseline_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    baseline_model.fit(X, y, epochs=epochs, batch_size=batch_size, validation_split=0.1)
    baseline_model.save_weights(os.path.join(data_dir, 'model.weights.h5'))
    return baseline_model

# Load noise data (if needed for baseline model training)
def load_noise_data():
    try:
        noise_data = np.load(os.path.join(data_dir, 'dataset_noise_pure_182.npy'))
        return noise_data
    except Exception as e:
        print(f"Lỗi khi tải dữ liệu nhiễu: {e}")
        return np.array([])

# Load or train baseline model
baseline_model = create_baseline_model(input_shape=880)
try:
    baseline_model.load_weights(os.path.join(data_dir, 'model.weights.h5'))
    print("Trọng số mô hình baseline đã được tải thành công!")
except Exception as e:
    print(f"Lỗi khi tải trọng số: {e}. Training baseline model.")
    noise_data = load_noise_data()
    if noise_data.size == 0:
        raise FileNotFoundError("Không thể tải dữ liệu nhiễu.")
    baseline_model = train_baseline_model(baseline_model, noise_data)

# Define DenseNet models
def build_1d_densenet(input_shape=(880, 1), num_classes=11, growth_rate=12):
    inputs = layers.Input(shape=input_shape)
    x = layers.Conv1D(48, 7, padding='same', activation='relu', kernel_regularizer=regularizers.l2(0.0005))(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling1D(pool_size=2)(x)
    def dense_block(x, num_layers, filters):
        for _ in range(num_layers):
            y = layers.BatchNormalization()(x)
            y = layers.Activation('relu')(y)
            y = layers.Conv1D(filters, 3, padding='same', kernel_regularizer=regularizers.l2(0.0005))(y)
            x = layers.Concatenate()([x, y])
        return x
    def transition_layer(x):
        filters = x.shape[-1]
        x = layers.BatchNormalization()(x)
        x = layers.Activation('relu')(x)
        x = layers.Conv1D(filters // 2, 1, padding='same', kernel_regularizer=regularizers.l2(0.0005))(x)
        x = layers.MaxPooling1D(pool_size=2)(x)
        return x
    for _ in range(3):
        x = dense_block(x, num_layers=4, filters=growth_rate)
        x = transition_layer(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dropout(0.4)(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)
    return models.Model(inputs, outputs)

def build_2d_densenet(input_shape=(64, 64, 1), num_classes=11, growth_rate=12):
    inputs = layers.Input(shape=input_shape)
    x = layers.Conv2D(48, 3, padding='same', activation='relu', kernel_regularizer=regularizers.l2(0.0005))(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)
    def dense_block(x, num_layers, filters):
        for _ in range(num_layers):
            y = layers.BatchNormalization()(x)
            y = layers.Activation('relu')(y)
            y = layers.Conv2D(filters, 3, padding='same', kernel_regularizer=regularizers.l2(0.0005))(y)
            x = layers.Concatenate()([x, y])
        return x
    def transition_layer(x):
        filters = x.shape[-1]
        x = layers.BatchNormalization()(x)
        x = layers.Activation('relu')(x)
        x = layers.Conv2D(filters // 2, 1, padding='same', kernel_regularizer=regularizers.l2(0.0005))(x)
        x = layers.MaxPooling2D(pool_size=(2, 2))(x)
        return x
    for _ in range(3):
        x = dense_block(x, num_layers=4, filters=growth_rate)
        x = transition_layer(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dropout(0.4)(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)
    return models.Model(inputs, outputs)

# Define ResNet models
def build_1d_resnet(input_shape=(880, 1), num_classes=11):
    inputs = layers.Input(shape=input_shape)
    x = layers.Conv1D(64, 5, padding='same', activation='relu', kernel_regularizer=regularizers.l2(0.0001))(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling1D(pool_size=2)(x)
    def residual_block(x, filters, kernel_size=3):
        shortcut = x
        x = layers.Conv1D(filters, kernel_size, padding='same', activation='relu', kernel_regularizer=regularizers.l2(0.0001))(x)
        x = layers.BatchNormalization()(x)
        x = layers.Conv1D(filters, kernel_size, padding='same', activation='relu', kernel_regularizer=regularizers.l2(0.0001))(x)
        x = layers.BatchNormalization()(x)
        if shortcut.shape[-1] != filters:
            shortcut = layers.Conv1D(filters, 1, padding='same')(shortcut)
        x = layers.Add()([shortcut, x])
        x = layers.Activation('relu')(x)
        return x
    x = residual_block(x, 64)
    x = residual_block(x, 64)
    x = layers.MaxPooling1D(pool_size=2)(x)
    x = residual_block(x, 128)
    x = residual_block(x, 128)
    x = residual_block(x, 128)
    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dropout(0.3)(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)
    return models.Model(inputs, outputs)

def build_2d_resnet(input_shape=(64, 64, 1), num_classes=11):
    inputs = layers.Input(shape=input_shape)
    x = layers.Conv2D(32, 3, padding='same', activation='relu', kernel_regularizer=regularizers.l2(0.0005))(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)
    def residual_block(x, filters, kernel_size=3):
        shortcut = x
        x = layers.Conv2D(filters, kernel_size, padding='same', activation='relu', kernel_regularizer=regularizers.l2(0.0005))(x)
        x = layers.BatchNormalization()(x)
        x = layers.Conv2D(filters, kernel_size, padding='same')(x)
        x = layers.BatchNormalization()(x)
        if shortcut.shape[-1] != filters:
            shortcut = layers.Conv2D(filters, 1, padding='same')(shortcut)
        x = layers.Add()([shortcut, x])
        x = layers.Activation('relu')(x)
        return x
    x = residual_block(x, 32)
    x = residual_block(x, 32)
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)
    return models.Model(inputs, outputs)

# Plot confusion matrix (updated version)
def plot_confusion_matrix(y_true, y_pred, title, filename):
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(12, 10))
    sns.heatmap(cm, annot=True, fmt='.2f', cmap='viridis', cbar=True,  # Changed to normalized values and viridis colormap
                xticklabels=[f"{i*10}% Ethanol" for i in range(11)],
                yticklabels=[f"{i*10}% Ethanol" for i in range(11)],
                annot_kws={"size": 10}, norm=plt.Normalize(vmin=0, vmax=np.max(cm)))
    plt.title(title, fontsize=14, pad=15)
    plt.xlabel('Predicted Label', fontsize=12)
    plt.ylabel('True Label', fontsize=12)
    plt.savefig(os.path.join(visualizations_dir, filename), dpi=300, bbox_inches='tight')
    plt.close()

# Data augmentation
data_augmentation_1d = models.Sequential([
    layers.Lambda(lambda x: x + tf.random.normal(tf.shape(x), mean=0.0, stddev=0.05)),
    layers.Lambda(lambda x: x * tf.random.uniform((), 0.8, 1.2)),
    layers.Lambda(lambda x: tf.roll(x, shift=tf.random.uniform((), -5, 5, dtype=tf.int32), axis=1))
])
data_augmentation_2d = models.Sequential([
    layers.Lambda(lambda x: x + tf.random.normal(tf.shape(x), mean=0.0, stddev=0.05)),
    layers.Lambda(lambda x: x * tf.random.uniform((), 0.8, 1.2)),
    layers.Lambda(lambda x: tf.roll(x, shift=tf.random.uniform((), -5, 5, dtype=tf.int32), axis=1))
])

# Split data
X_1d_train, X_1d_test, y_train, y_test = train_test_split(X_1d, y, test_size=0.2, random_state=42)
X_2d_train, X_2d_test, y_train_2d, y_test_2d = train_test_split(X_2d, y, test_size=0.2, random_state=42)

# Compute class weights
class_weights = compute_class_weight('balanced', classes=np.arange(11), y=y)
class_weight = {i: w for i, w in enumerate(class_weights)}

# Train models with fresh optimizers
early_stopping = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)

# DenseNet 1D
tf.keras.backend.clear_session()
lr_schedule_1d_densenet = keras.optimizers.schedules.CosineDecay(initial_learning_rate=0.001, decay_steps=10*len(X_1d_train)//64)
optimizer_1d_densenet = keras.optimizers.Adam(learning_rate=lr_schedule_1d_densenet)
densenet_1d = models.Sequential([data_augmentation_1d, build_1d_densenet()])
densenet_1d.compile(optimizer=optimizer_1d_densenet, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
densenet_1d.fit(X_1d_train, y_train, validation_split=0.1, epochs=10, batch_size=64,
                callbacks=[keras.callbacks.ModelCheckpoint(os.path.join(model_dir, "best_densenet_1d.keras"), save_best_only=True), early_stopping],
                class_weight=class_weight)

# DenseNet 2D
tf.keras.backend.clear_session()
lr_schedule_2d_densenet = keras.optimizers.schedules.CosineDecay(initial_learning_rate=0.001, decay_steps=10*len(X_2d_train)//32)
optimizer_2d_densenet = keras.optimizers.Adam(learning_rate=lr_schedule_2d_densenet)
densenet_2d = models.Sequential([data_augmentation_2d, build_2d_densenet()])
densenet_2d.compile(optimizer=optimizer_2d_densenet, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
densenet_2d.fit(X_2d_train, y_train_2d, validation_split=0.1, epochs=10, batch_size=32,
                callbacks=[keras.callbacks.ModelCheckpoint(os.path.join(model_dir, "best_densenet_2d.keras"), save_best_only=True), early_stopping],
                class_weight=class_weight)

# ResNet 1D
tf.keras.backend.clear_session()
lr_schedule_1d_resnet = keras.optimizers.schedules.CosineDecay(initial_learning_rate=0.001, decay_steps=10*len(X_1d_train)//64)
optimizer_1d_resnet = keras.optimizers.Adam(learning_rate=lr_schedule_1d_resnet)
resnet_1d = models.Sequential([data_augmentation_1d, build_1d_resnet()])
resnet_1d.compile(optimizer=optimizer_1d_resnet, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
resnet_1d.fit(X_1d_train, y_train, validation_split=0.1, epochs=10, batch_size=64,
              callbacks=[keras.callbacks.ModelCheckpoint(os.path.join(model_dir, "best_resnet_1d.keras"), save_best_only=True), early_stopping],
              class_weight=class_weight)

# ResNet 2D
tf.keras.backend.clear_session()
lr_schedule_2d_resnet = keras.optimizers.schedules.CosineDecay(initial_learning_rate=0.001, decay_steps=10*len(X_2d_train)//32)
optimizer_2d_resnet = keras.optimizers.Adam(learning_rate=lr_schedule_2d_resnet)
resnet_2d = models.Sequential([data_augmentation_2d, build_2d_resnet()])
resnet_2d.compile(optimizer=optimizer_2d_resnet, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
resnet_2d.fit(X_2d_train, y_train_2d, validation_split=0.1, epochs=10, batch_size=32,
              callbacks=[keras.callbacks.ModelCheckpoint(os.path.join(model_dir, "best_resnet_2d.keras"), save_best_only=True), early_stopping],
              class_weight=class_weight)

# Evaluate
y_pred_1d_densenet = densenet_1d.predict(X_1d_test)
y_pred_2d_densenet = densenet_2d.predict(X_2d_test)
y_pred_1d_resnet = resnet_1d.predict(X_1d_test)
y_pred_2d_resnet = resnet_2d.predict(X_2d_test)

y_pred_1d_densenet_labels = np.argmax(y_pred_1d_densenet, axis=1)
y_pred_2d_densenet_labels = np.argmax(y_pred_2d_densenet, axis=1)
y_pred_1d_resnet_labels = np.argmax(y_pred_1d_resnet, axis=1)
y_pred_2d_resnet_labels = np.argmax(y_pred_2d_resnet, axis=1)

densenet_1d_acc = np.mean(y_pred_1d_densenet_labels == y_test)
densenet_2d_acc = np.mean(y_pred_2d_densenet_labels == y_test_2d)
resnet_1d_acc = np.mean(y_pred_1d_resnet_labels == y_test)
resnet_2d_acc = np.mean(y_pred_2d_resnet_labels == y_test_2d)

densenet_1d_precision = precision_score(y_test, y_pred_1d_densenet_labels, average='macro')
densenet_2d_precision = precision_score(y_test_2d, y_pred_2d_densenet_labels, average='macro')
resnet_1d_precision = precision_score(y_test, y_pred_1d_resnet_labels, average='macro')
resnet_2d_precision = precision_score(y_test_2d, y_pred_2d_resnet_labels, average='macro')

densenet_1d_recall = recall_score(y_test, y_pred_1d_densenet_labels, average='macro')
densenet_2d_recall = recall_score(y_test_2d, y_pred_2d_densenet_labels, average='macro')
resnet_1d_recall = recall_score(y_test, y_pred_1d_resnet_labels, average='macro')
resnet_2d_recall = recall_score(y_test_2d, y_pred_2d_resnet_labels, average='macro')

densenet_1d_f1 = f1_score(y_test, y_pred_1d_densenet_labels, average='macro')
densenet_2d_f1 = f1_score(y_test_2d, y_pred_2d_densenet_labels, average='macro')
resnet_1d_f1 = f1_score(y_test, y_pred_1d_resnet_labels, average='macro')
resnet_2d_f1 = f1_score(y_test_2d, y_pred_2d_resnet_labels, average='macro')

print("\nKết quả đánh giá:")
print(f"DenseNet 1D - Accuracy: {densenet_1d_acc:.4f}, Precision: {densenet_1d_precision:.4f}, Recall: {densenet_1d_recall:.4f}, F1: {densenet_1d_f1:.4f}")
print(f"DenseNet 2D (GADF) - Accuracy: {densenet_2d_acc:.4f}, Precision: {densenet_2d_precision:.4f}, Recall: {densenet_2d_recall:.4f}, F1: {densenet_2d_f1:.4f}")
print(f"ResNet 1D - Accuracy: {resnet_1d_acc:.4f}, Precision: {resnet_1d_precision:.4f}, Recall: {resnet_1d_recall:.4f}, F1: {resnet_1d_f1:.4f}")
print(f"ResNet 2D (GADF) - Accuracy: {resnet_2d_acc:.4f}, Precision: {resnet_2d_precision:.4f}, Recall: {resnet_2d_recall:.4f}, F1: {resnet_2d_f1:.4f}")

# Save results
plot_confusion_matrix(y_test, y_pred_1d_densenet_labels, "Confusion Matrix - DenseNet 1D", "cm_densenet_1d.png")
plot_confusion_matrix(y_test_2d, y_pred_2d_densenet_labels, "Confusion Matrix - DenseNet 2D (GADF)", "cm_densenet_2d_gadf.png")
plot_confusion_matrix(y_test, y_pred_1d_resnet_labels, "Confusion Matrix - ResNet 1D", "cm_resnet_1d.png")
plot_confusion_matrix(y_test_2d, y_pred_2d_resnet_labels, "Confusion Matrix - ResNet 2D (GADF)", "cm_resnet_2d_gadf.png")

densenet_1d.save(os.path.join(model_dir, 'densenet_1d_full.keras'))
densenet_2d.save(os.path.join(model_dir, 'densenet_2d_full.keras'))
resnet_1d.save(os.path.join(model_dir, 'resnet_1d_full.keras'))
resnet_2d.save(os.path.join(model_dir, 'resnet_2d_full.keras'))

np.save(os.path.join(results_dir, 'y_pred_1d_densenet.npy'), y_pred_1d_densenet)
np.save(os.path.join(results_dir, 'y_pred_2d_densenet.npy'), y_pred_2d_densenet)
np.save(os.path.join(results_dir, 'y_pred_1d_resnet.npy'), y_pred_1d_resnet)
np.save(os.path.join(results_dir, 'y_pred_2d_resnet.npy'), y_pred_2d_resnet)

metrics = {
    'densenet_1d': {'accuracy': densenet_1d_acc, 'precision': densenet_1d_precision, 'recall': densenet_1d_recall, 'f1': densenet_1d_f1},
    'densenet_2d': {'accuracy': densenet_2d_acc, 'precision': densenet_2d_precision, 'recall': densenet_2d_recall, 'f1': densenet_2d_f1},
    'resnet_1d': {'accuracy': resnet_1d_acc, 'precision': resnet_1d_precision, 'recall': resnet_1d_recall, 'f1': resnet_1d_f1},
    'resnet_2d': {'accuracy': resnet_2d_acc, 'precision': resnet_2d_precision, 'recall': resnet_2d_recall, 'f1': resnet_2d_f1}
}
with open(os.path.join(results_dir, 'metrics.json'), 'w') as f:
    json.dump(metrics, f, indent=4)

# Visualization

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

# Set matplotlib font to support superscript minus
plt.rcParams['font.family'] = 'DejaVu Sans'

# =======================================================
# Directories
# =======================================================
data_dir = 'data'
visualizations_dir = os.path.join(data_dir, 'visualizations')
experiment_dir = os.path.join('experiments', 'experiment_20250914_220130')  # Fixed timestamp
results_dir = os.path.join(experiment_dir, 'results')
labels_dir = os.path.join(data_dir, 'labels')

# Create visualizations directory if it doesn't exist
os.makedirs(visualizations_dir, exist_ok=True)

# =======================================================
# Load test labels
# =======================================================
try:
    labels_df = pd.read_csv(os.path.join(labels_dir, 'labels.csv'))
    y = labels_df['label'].values
    _, y_test = train_test_split(y, test_size=0.2, random_state=42)
    print("Test labels loaded successfully!")
except Exception as e:
    print("Error loading labels:", str(e))
    raise

# Convert ethanol labels to methanol (100 - ethanol%)
y_test_methanol = 100 - y_test * 10  # Assuming y_test is in range 0-10 for 0-100%

# =======================================================
# Load predictions
# =======================================================
try:
    y_pred_1d_densenet = np.load(os.path.join(results_dir, 'y_pred_1d_densenet.npy'))
    y_pred_2d_densenet = np.load(os.path.join(results_dir, 'y_pred_2d_densenet.npy'))
    y_pred_1d_resnet = np.load(os.path.join(results_dir, 'y_pred_1d_resnet.npy'))
    y_pred_2d_resnet = np.load(os.path.join(results_dir, 'y_pred_2d_resnet.npy'))
    print("All prediction files loaded successfully!")
except FileNotFoundError as e:
    print("Error: One or more prediction files not found:", str(e))
    print("Please ensure all files (y_pred_1d_densenet.npy, y_pred_2d_densenet.npy, "
          "y_pred_1d_resnet.npy, y_pred_2d_resnet.npy) exist in", results_dir)
    raise

# Convert predictions to labels
y_pred_1d_densenet_labels = np.argmax(y_pred_1d_densenet, axis=1)
y_pred_2d_densenet_labels = np.argmax(y_pred_2d_densenet, axis=1)
y_pred_1d_resnet_labels = np.argmax(y_pred_1d_resnet, axis=1)
y_pred_2d_resnet_labels = np.argmax(y_pred_2d_resnet, axis=1)

# Convert predicted ethanol labels to methanol (100 - ethanol%)
y_pred_1d_densenet_labels_methanol = 100 - y_pred_1d_densenet_labels * 10
y_pred_2d_densenet_labels_methanol = 100 - y_pred_2d_densenet_labels * 10
y_pred_1d_resnet_labels_methanol = 100 - y_pred_1d_resnet_labels * 10
y_pred_2d_resnet_labels_methanol = 100 - y_pred_2d_resnet_labels * 10

# =======================================================
# Normalized confusion matrix function
# =======================================================
def calculate_normalized_confusion_matrix(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred)
    cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis] * 100
    cm_normalized = np.nan_to_num(cm_normalized, nan=0.0)
    return cm_normalized

def plot_normalized_confusion_matrix(y_true, y_pred, title, filename):
    cm_normalized = calculate_normalized_confusion_matrix(y_true, y_pred)
    labels = [f"{i}%" for i in range(0, 101, 10)][::-1]  # Reversed for methanol (100% to 0%)

    plt.figure(figsize=(12, 10))
    sns.heatmap(cm_normalized, annot=True, fmt='.1f', cmap='Blues',
                xticklabels=labels, yticklabels=labels, cbar_kws={'label': '%'},
                annot_kws={'size': 14},  # Increase annotation font size
                square=True)  # Ensure square cells
    plt.xlabel('Predicted Methanol Ratio', fontsize=18, weight='bold')
    plt.ylabel('True Methanol Ratio', fontsize=18, weight='bold')
    plt.title(title, fontsize=20, weight='bold', pad=20)
    plt.xticks(fontsize=14, rotation=45, ha='right')
    plt.yticks(fontsize=14, rotation=0)
    cbar = plt.gcf().axes[-1]
    cbar.set_ylabel('%', fontsize=16, weight='bold')
    cbar.tick_params(labelsize=14)
    plt.tight_layout()
    plt.savefig(os.path.join(visualizations_dir, filename), dpi=300, bbox_inches='tight')
    plt.close()

# =======================================================
# Plot normalized confusion matrices for all models
# =======================================================
# DenseNet 1D
plot_normalized_confusion_matrix(
    y_test_methanol,
    y_pred_1d_densenet_labels_methanol,
    "Normalized Confusion Matrix for DenseNet 1D (%)",
    "normalized_confusion_matrix_densenet_1d.png"
)

# DenseNet 2D
plot_normalized_confusion_matrix(
    y_test_methanol,
    y_pred_2d_densenet_labels_methanol,
    "Normalized Confusion Matrix for DenseNet 2D (%)",
    "normalized_confusion_matrix_densenet_2d.png"
)

# ResNet 1D
plot_normalized_confusion_matrix(
    y_test_methanol,
    y_pred_1d_resnet_labels_methanol,
    "Normalized Confusion Matrix for ResNet 1D (%)",
    "normalized_confusion_matrix_resnet_1d.png"
)

# ResNet 2D
plot_normalized_confusion_matrix(
    y_test_methanol,
    y_pred_2d_resnet_labels_methanol,
    "Normalized Confusion Matrix for ResNet 2D (%)",
    "normalized_confusion_matrix_resnet_2d.png"
)

# =======================================================
# Print normalized confusion matrix values
# =======================================================
models = [
    ('DenseNet 1D', y_pred_1d_densenet_labels_methanol),
    ('DenseNet 2D', y_pred_2d_densenet_labels_methanol),
    ('ResNet 1D', y_pred_1d_resnet_labels_methanol),
    ('ResNet 2D', y_pred_2d_resnet_labels_methanol)
]

for model_name, y_pred_labels in models:
    cm_normalized = calculate_normalized_confusion_matrix(y_test_methanol, y_pred_labels)
    print(f"\nNormalized Confusion Matrix for {model_name} (%):")
    for i, row in enumerate(cm_normalized):
        print(f"True {100 - i*10}% Methanol:")
        for j, value in enumerate(row):
            print(f"  Predicted {100 - j*10}%: {value:.1f}%")

Test labels loaded successfully!
All prediction files loaded successfully!

Normalized Confusion Matrix for DenseNet 1D (%):
True 100% Methanol:
  Predicted 100%: 91.7%
  Predicted 90%: 7.8%
  Predicted 80%: 0.5%
  Predicted 70%: 0.0%
  Predicted 60%: 0.0%
  Predicted 50%: 0.0%
  Predicted 40%: 0.0%
  Predicted 30%: 0.0%
  Predicted 20%: 0.0%
  Predicted 10%: 0.0%
  Predicted 0%: 0.0%
True 90% Methanol:
  Predicted 100%: 3.8%
  Predicted 90%: 92.9%
  Predicted 80%: 3.3%
  Predicted 70%: 0.0%
  Predicted 60%: 0.0%
  Predicted 50%: 0.0%
  Predicted 40%: 0.0%
  Predicted 30%: 0.0%
  Predicted 20%: 0.0%
  Predicted 10%: 0.0%
  Predicted 0%: 0.0%
True 80% Methanol:
  Predicted 100%: 0.9%
  Predicted 90%: 7.9%
  Predicted 80%: 90.2%
  Predicted 70%: 0.9%
  Predicted 60%: 0.0%
  Predicted 50%: 0.0%
  Predicted 40%: 0.0%
  Predicted 30%: 0.0%
  Predicted 20%: 0.0%
  Predicted 10%: 0.0%
  Predicted 0%: 0.0%
True 70% Methanol:
  Predicted 100%: 0.0%
  Predicted 90%: 0.5%
  Predicted 80%: 4.1%
  

# Occasional Analysis

In [50]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import keras
from keras.layers import Layer
from keras import layers, models, optimizers, regularizers
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
from sklearn.metrics import accuracy_score
import seaborn as sns
from datetime import datetime

# Set matplotlib font to support superscript minus
plt.rcParams['font.family'] = 'DejaVu Sans'

# =======================================================
# Custom layers to replace Lambda layers
# =======================================================
class L2NormLayer(Layer):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)

    def call(self, inputs):
        return tf.math.l2_normalize(inputs, axis=1)

    def get_config(self):
        return super().get_config()

class ReduceMeanLayer(Layer):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)

    def call(self, inputs):
        return tf.reduce_mean(inputs, axis=1)

    def get_config(self):
        return super().get_config()

class DataAugmentation1D(Layer):
    def __init__(self, noise_factor=0.08, **kwargs):  # Reduced noise_factor
        super().__init__(**kwargs)
        self.noise_factor = noise_factor

    def call(self, inputs, training=None):
        if training:
            noise = tf.random.normal(shape=tf.shape(inputs), mean=0.0,
                                     stddev=self.noise_factor, dtype=inputs.dtype)
            return inputs + noise
        return inputs

    def get_config(self):
        config = super().get_config()
        config.update({"noise_factor": self.noise_factor})
        return config

class DataAugmentation2D(Layer):
    def __init__(self, noise_factor=0.08, **kwargs):
        super().__init__(**kwargs)
        self.noise_factor = noise_factor

    def call(self, inputs, training=None):
        if training:
            noise = tf.random.normal(shape=tf.shape(inputs), mean=0.0,
                                     stddev=self.noise_factor, dtype=inputs.dtype)
            return inputs + noise
        return inputs

    def get_config(self):
        config = super().get_config()
        config.update({"noise_factor": self.noise_factor})
        return config

# =======================================================
# DenseNet-1D architecture with balanced regularization
# =======================================================
def build_1d_densenet(input_shape, num_classes):
    inputs = layers.Input(shape=input_shape)

    # Initial conv block
    x = DataAugmentation1D(0.08)(inputs)
    x = layers.Conv1D(64, 7, strides=2, padding='same', activation='relu',
                      kernel_regularizer=regularizers.l2(0.01))(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling1D(pool_size=3, strides=2, padding='same')(x)
    x = layers.Dropout(0.1)(x)  # Reduced dropout

    # Dense block
    for _ in range(6):
        y = layers.BatchNormalization()(x)
        y = layers.Activation('relu')(y)
        y = layers.Conv1D(32, 3, padding='same', kernel_regularizer=regularizers.l2(0.01))(y)
        y = layers.Dropout(0.1)(y)  # Reduced dropout
        x = layers.Concatenate()([x, y])

    # Transition
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.Conv1D(128, 1, padding='same', kernel_regularizer=regularizers.l2(0.01))(x)
    x = layers.AveragePooling1D(pool_size=2, strides=2, padding='same')(x)
    x = layers.Dropout(0.1)(x)  # Reduced dropout

    # Global features
    x = ReduceMeanLayer(name="reduce_mean")(x)
    x = L2NormLayer(name="l2_norm")(x)

    outputs = layers.Dense(num_classes, activation='softmax')(x)

    model = models.Model(inputs, outputs, name='DenseNet1D')
    return model

# =======================================================
# Directories
# =======================================================
data_dir = 'data'
synthetic_dir = os.path.join(data_dir, 'synthetic')
labels_dir = os.path.join(data_dir, 'labels')
visualizations_dir = os.path.join(data_dir, 'visualizations')
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
experiment_dir = os.path.join('experiments', f'experiment_{timestamp}')
model_dir = os.path.join(experiment_dir, 'models')
os.makedirs(model_dir, exist_ok=True)
os.makedirs(visualizations_dir, exist_ok=True)

# =======================================================
# Load data
# =======================================================
try:
    X_1d = np.load(os.path.join(synthetic_dir, 'synthetic_1d.npy'))
    labels_df = pd.read_csv(os.path.join(labels_dir, 'labels.csv'))
    y = labels_df['label'].values
    print("Data loaded successfully!")
except Exception as e:
    print("Error loading data:", str(e))
    raise

# Plot sample spectra to inspect for artifacts
wavenumbers = np.linspace(500, 3500, 880)
plt.figure(figsize=(10, 6))
for i in range(5):  # Plot 5 random spectra
    idx = np.random.randint(0, X_1d.shape[0])
    plt.plot(wavenumbers, X_1d[idx, :, 0], label=f'Sample {i+1} (Label {y[idx]})')
plt.xlabel('Wavenumber (cm⁻¹)')
plt.ylabel('Intensity')
plt.title('Sample Raman Spectra')
plt.legend()
plt.grid(True)
plt.savefig(os.path.join(visualizations_dir, 'sample_spectra.png'), dpi=300)
plt.close()

# Split data
X_train, X_test, y_train, y_test = train_test_split(X_1d, y, test_size=0.2, random_state=42)
y_train_cat = keras.utils.to_categorical(y_train, num_classes=11)
y_test_cat = keras.utils.to_categorical(y_test, num_classes=11)

# =======================================================
# Build and train model
# =======================================================
densenet_1d = build_1d_densenet(input_shape=(880, 1), num_classes=11)
densenet_1d.compile(optimizer=optimizers.Adam(1e-3),
                    loss='categorical_crossentropy',
                    metrics=['accuracy'])

# Train
densenet_1d.fit(X_train, y_train_cat,
                validation_data=(X_test, y_test_cat),
                epochs=50,  # Increased for better convergence
                batch_size=32,
                verbose=1)

# Save model
model_path = os.path.join(model_dir, 'densenet_1d_full.keras')
densenet_1d.save(model_path)
print(f"Model saved to {model_path}")

# =======================================================
# Load model for occlusion analysis
# =======================================================
custom_objects = {
    "DataAugmentation1D": DataAugmentation1D,
    "DataAugmentation2D": DataAugmentation2D,
    "L2NormLayer": L2NormLayer,
    "ReduceMeanLayer": ReduceMeanLayer
}
try:
    densenet_1d = models.load_model(model_path, custom_objects=custom_objects)
    print("Model loaded successfully!")
except Exception as e:
    print("Error loading model:", str(e))
    print("Possible issues:")
    print("- Model file corrupted or saved with incompatible Keras version.")
    print("- Ensure custom layers (L2NormLayer, ReduceMeanLayer) are defined correctly.")
    raise

# =======================================================
# Occlusion analysis with 20 regions
# =======================================================
# Select 550 spectra for occlusion (stratified, ~50 per class)
sss = StratifiedShuffleSplit(n_splits=1, test_size=550, random_state=42)
for _, test_idx in sss.split(X_test, y_test):
    X_occlusion = X_test[test_idx]
    y_occlusion = y_test[test_idx]

print("Occlusion test set shape:", X_occlusion.shape)
print("Label distribution:\n", pd.Series(y_occlusion).value_counts())

def occlusion_analysis(model, X, y, window_size=44, wavenumbers=None):
    num_windows = X.shape[1] // window_size  # 20 windows
    windows = [(i * window_size, (i + 1) * window_size) for i in range(num_windows)]
    window_labels = [f"W{i} ({int(wavenumbers[start])}-{int(wavenumbers[end-1])} cm⁻¹)"
                     for i, (start, end) in enumerate(windows)]

    # Original accuracy
    y_pred = model.predict(X, verbose=0)
    y_pred_labels = np.argmax(y_pred, axis=1)
    original_accuracy = accuracy_score(y, y_pred_labels)
    print(f"Original Accuracy: {original_accuracy:.4f}")

    # Occlusion
    occlusion_accuracies = np.zeros(num_windows)
    accuracy_drops = np.zeros(num_windows)

    for win_idx, (start, end) in enumerate(windows):
        X_occluded = X.copy()
        X_occluded[:, start:end, :] = 0  # Occlude by setting to 0
        y_pred_occluded = model.predict(X_occluded, verbose=0)
        y_pred_occluded_labels = np.argmax(y_pred_occluded, axis=1)
        acc = accuracy_score(y, y_pred_occluded_labels)
        occlusion_accuracies[win_idx] = acc
        accuracy_drops[win_idx] = original_accuracy - acc
        print(f"{window_labels[win_idx]}: Acc = {acc:.4f}, Drop = {accuracy_drops[win_idx]:.4f}")

    # Per-class analysis with console output
    unique_labels = sorted(np.unique(y))
    group_accuracy_drops = np.zeros((len(unique_labels), num_windows))
    print("\nPer-class Accuracy Drops:")
    for grp_idx, label in enumerate(unique_labels):
        mask = y == label
        X_group = X[mask]
        y_group = y[mask]
        y_pred_group = model.predict(X_group, verbose=0)
        orig_acc_group = accuracy_score(y_group, np.argmax(y_pred_group, axis=1))
        print(f"\nClass {label} ({label*10}% Ethanol / {(10-label)*10}% Methanol): Original Acc = {orig_acc_group:.4f}")
        for win_idx, (start, end) in enumerate(windows):
            X_occluded_group = X_group.copy()
            X_occluded_group[:, start:end, :] = 0
            y_pred_occ_group = model.predict(X_occluded_group, verbose=0)
            acc_group = accuracy_score(y_group, np.argmax(y_pred_occ_group, axis=1))
            group_accuracy_drops[grp_idx, win_idx] = orig_acc_group - acc_group
            print(f"  {window_labels[win_idx]}: Acc = {acc_group:.4f}, Drop = {group_accuracy_drops[grp_idx, win_idx]:.4f}")

    # Summary of top windows per class
    print("\nTop Windows per Class:")
    for grp_idx, label in enumerate(unique_labels):
        top_windows = np.argsort(group_accuracy_drops[grp_idx])[::-1][:3]  # Top 3 windows
        top_drops = group_accuracy_drops[grp_idx, top_windows]
        print(f"Class {label} ({label*10}% Ethanol / {(10-label)*10}% Methanol):")
        for i, win_idx in enumerate(top_windows):
            print(f"  {window_labels[win_idx]}: Drop = {top_drops[i]:.4f}")

    return accuracy_drops, group_accuracy_drops, window_labels, unique_labels

# Run occlusion analysis
wavenumbers = np.linspace(500, 3500, 880)
accuracy_drops, group_accuracy_drops, window_labels, unique_labels = occlusion_analysis(
    densenet_1d, X_occlusion, y_occlusion, window_size=44, wavenumbers=wavenumbers
)

# =======================================================
# Plotting
# =======================================================
# Plot 1: Bar chart of accuracy drops
plt.figure(figsize=(14, 6))
plt.bar(window_labels, accuracy_drops * 100, color='skyblue')
plt.xlabel('Occluded Window (Wavenumber Range)')
plt.ylabel('Accuracy Drop (%)')
plt.title('Accuracy Drop When Occluding Each Window (DenseNet 1D)')
plt.xticks(rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.7)
ethanol_idx = 3  # W3 contains ~870-890 cm⁻¹ (851-998 cm⁻¹)
methanol_idx = 4  # W4 contains ~1000-1020 cm⁻¹ (1002-1149 cm⁻¹)
plt.axvspan(ethanol_idx - 0.5, ethanol_idx + 0.5, color='green', alpha=0.1, label='Ethanol Peak (~870-890 cm⁻¹)')
plt.axvspan(methanol_idx - 0.5, methanol_idx + 0.5, color='red', alpha=0.1, label='Methanol Peak (~1000-1020 cm⁻¹)')
plt.legend()
plt.tight_layout()
plt.savefig(os.path.join(visualizations_dir, 'occlusion_accuracy_drop.png'), dpi=300)
plt.close()

# Plot 2: Heatmap of per-class accuracy drops
ratio_labels = [f"{i*10}% Ethanol / {(10-i)*10}% Methanol" for i in unique_labels]
plt.figure(figsize=(14, 8))
sns.heatmap(group_accuracy_drops * 100, annot=True, fmt='.2f', cmap='Reds',
            xticklabels=window_labels, yticklabels=ratio_labels,
            cbar_kws={'label': 'Accuracy Drop (%)'})
plt.xlabel('Occluded Window')
plt.ylabel('Ethanol/Methanol Ratio')
plt.title('Accuracy Drop (%) per Window and Ratio (Red = Important Region)')
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig(os.path.join(visualizations_dir, 'occlusion_heatmap_per_ratio.png'), dpi=300)
plt.close()

# =======================================================
# Interpretation
# =======================================================
print("\nInterpretation:")
print("- High accuracy drop in a window → That region is important for the model.")
print("- Expect high drops in W3 (851-998 cm⁻¹) for high Ethanol ratios (labels 6-10) and W4 (1002-1149 cm⁻¹) for high Methanol ratios (labels 0-5).")
print("- Check the heatmap: Red regions in W3/W4 indicate reliance on physically meaningful features.")
print("- If high drops occur in non-characteristic windows (e.g., W17: 2902-3049 cm⁻¹), inspect the dataset for artifacts (see sample_spectra.png).")
print("- Consider increasing epochs or adjusting regularization if W3/W4 drops are low.")

Data loaded successfully!
Epoch 1/50
275/275 ━━━━━━━━━━━━━━━━━━━━ 48s 147ms/step - accuracy: 0.1826 - loss: 4.0311 - val_accuracy: 0.0918 - val_loss: 3.1717
Epoch 2/50
275/275 ━━━━━━━━━━━━━━━━━━━━ 36s 132ms/step - accuracy: 0.4230 - loss: 1.7904 - val_accuracy: 0.1423 - val_loss: 3.2719
Epoch 3/50
275/275 ━━━━━━━━━━━━━━━━━━━━ 37s 133ms/step - accuracy: 0.4735 - loss: 1.5572 - val_accuracy: 0.2114 - val_loss: 2.0559
Epoch 4/50
275/275 ━━━━━━━━━━━━━━━━━━━━ 37s 133ms/step - accuracy: 0.5240 - loss: 1.4284 - val_accuracy: 0.2327 - val_loss: 2.1822
Epoch 5/50
275/275 ━━━━━━━━━━━━━━━━━━━━ 34s 124ms/step - accuracy: 0.5551 - loss: 1.3842 - val_accuracy: 0.2323 - val_loss: 1.9839
Epoch 6/50
275/275 ━━━━━━━━━━━━━━━━━━━━ 34s 122ms/step - accuracy: 0.5792 - loss: 1.2632 - val_accuracy: 0.3827 - val_loss: 1.6827
Epoch 7/50
275/275 ━━━━━━━━━━━━━━━━━━━━ 33s 119ms/step - accuracy: 0.6269 - loss: 1.1786 - val_accuracy: 0.5777 - val_loss: 1.2915
Epoch 8/50
275/275 ━━━━━━━━━━━━━━━━━━━━ 33s 118ms/step - 

# Compare performance with Baseline Removal to not have Baseline Removal

In [6]:
# -*- coding: utf-8 -*-
import os
import numpy as np
import pandas as pd
from scipy.interpolate import interp1d
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from pyts.transformation import GADF
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.utils.class_weight import compute_class_weight

# ==============================
# SEED & THƯ MỤC
# ==============================
np.random.seed(42)
tf.random.set_seed(42)

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
exp_dir = f'git s/NO_BASELINE_{timestamp}'
os.makedirs(exp_dir, exist_ok=True)
os.makedirs(f'{exp_dir}/models', exist_ok=True)
os.makedirs(f'{exp_dir}/results', exist_ok=True)

# ==============================
# LOAD DỮ LIỆU
# ==============================
df = pd.read_excel('data/Ethanol_Methanol.xlsx', usecols='A:L')

# Tên cột đúng trong file Excel của bạn
name_to_ratio = {
    'Ethanol': 1.0, 'Methanol': 0.0,
    'EM1_a': 0.9, 'EM2_a': 0.8, 'EM3_a': 0.7,
    'EM4_a': 0.6, 'EM5_a': 0.5, 'EM6_a': 0.4,
    'EM7_a': 0.3, 'EM8_a': 0.2, 'EM9_a': 0.1
}

# Hàm interpolate chung (chỉ dùng cho dữ liệu gốc)
def to_880(arr):
    arr = arr[~np.isnan(arr)]                     # bỏ NaN
    if len(arr) == 880:
        return arr
    x_old = np.linspace(0, len(arr)-1, len(arr))
    x_new = np.linspace(0, len(arr)-1, 880)
    f = interp1d(x_old, arr, kind='linear', fill_value='extrapolate')
    return f(x_new)

spectra_data = {name: to_880(df[name].values) for name in name_to_ratio}
noise_data   = np.load('data/dataset_noise_pure_182.npy')   # (N,1,2,880,1)

# ==============================
# HELPER
# ==============================
def normalize(s):
    s = s - s.min()
    return s / s.max() if s.max() > 0 else s

def generate_synthetic(clean_spec):
    idx = np.random.randint(len(noise_data))
    real_noise = noise_data[idx,0,1,:,0] - noise_data[idx,0,0,:,0]
    s = clean_spec + real_noise * np.random.uniform(1.0, 2.0)
    s += np.random.normal(0, 0.05*s.std(), 880)

    # vẫn thêm nền tổng hợp như cũ
    if np.random.rand() < 0.6:
        x = np.arange(880)
        if np.random.rand() < 0.5:                              # poly
            baseline = (x/879)**np.random.uniform(1.9, 2.1)
        else:                                                   # gaussian
            mu, sd = np.random.uniform(0,880), np.random.uniform(250,300)
            baseline = np.exp(-0.5*((x-mu)/sd)**2)
        s += baseline * np.random.uniform(0.7, 0.9)

    if np.random.rand() < 0.5:
        s = np.roll(s, np.random.randint(-10, 11))
    return s

def gadf_map(spec, size=64):
    spec = normalize(spec)
    spec = 2*spec - 1
    # cắt hoặc pad để chia hết cho 64
    target_len = size * (len(spec)//size)
    if len(spec) > target_len:
        spec = spec[:target_len]
    else:
        spec = np.pad(spec, (0, target_len-len(spec)), mode='constant')
    return GADF(image_size=size, overlapping=False, scale='-1')\
           .fit_transform(spec.reshape(1,-1))[0][:,:,np.newaxis]

# ==============================
# TẠO 11.000 PHỔ GADF (KHÔNG TRỪ NỀN)
# ==============================
X_2d = []
y    = []

# ánh xạ tỉ lệ → nhãn 0-10
ratio_to_label = {v: i for i, v in enumerate(sorted(name_to_ratio.values()))}

print("Đang tạo 11.000 phổ GADF (không trừ nền)...")
for name, ratio in name_to_ratio.items():
    clean = spectra_data[name]
    norm_clean = normalize(clean)

    # phổ gốc
    X_2d.append(gadf_map(norm_clean))
    y.append(ratio_to_label[ratio])

    # 999 phổ tổng hợp
    for i in range(999):
        synth = generate_synthetic(norm_clean)
        X_2d.append(gadf_map(normalize(synth)))
        y.append(ratio_to_label[ratio])

        if (i+1) % 400 == 0:
            print(f"   → {name}: {i+1}/999")

X_2d = np.array(X_2d)
y    = np.array(y)
print(f"\nHoàn tất! X_2d shape: {X_2d.shape}  |  Label distribution:\n{np.bincount(y)}")

# lưu tạm để lần sau không phải tạo lại
np.save(f'{exp_dir}/gadf_no_baseline.npy', X_2d)
np.save(f'{exp_dir}/labels_no_baseline.npy', y)

# ==============================
# MODEL 2D (DenseNet & ResNet)
# ==============================
def build_densenet_2d():
    inputs = layers.Input((64,64,1))
    x = layers.Conv2D(48,3,padding='same',activation='relu')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D(2)(x)
    for _ in range(3):
        for __ in range(4):
            y = layers.BatchNormalization()(x)
            y = layers.ReLU()(y)
            y = layers.Conv2D(12,3,padding='same')(y)
            x = layers.Concatenate()([x,y])
        x = layers.BatchNormalization()(x)
        x = layers.ReLU()(x)
        x = layers.Conv2D(x.shape[-1]//2,1,padding='same')(x)
        x = layers.MaxPooling2D(2)(x)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(128,activation='relu')(x)
    x = layers.Dropout(0.4)(x)
    outputs = layers.Dense(11,activation='softmax')(x)
    return models.Model(inputs, outputs)

def build_resnet_2d():
    inputs = layers.Input((64,64,1))
    x = layers.Conv2D(32,3,padding='same',activation='relu')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D(2)(x)
    def rb(x,f):
        s = x
        x = layers.Conv2D(f,3,padding='same',activation='relu')(x)
        x = layers.BatchNormalization()(x)
        x = layers.Conv2D(f,3,padding='same')(x)
        x = layers.BatchNormalization()(x)
        if s.shape[-1] != f:
            s = layers.Conv2D(f,1,padding='same')(s)
        x = layers.Add()([s,x])
        return layers.ReLU()(x)
    x = rb(x,32); x = rb(x,32)
    x = layers.MaxPooling2D(2)(x)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(128,activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(11,activation='softmax')(x)
    return models.Model(inputs, outputs)

aug = keras.Sequential([layers.RandomFlip("horizontal"),
                        layers.RandomRotation(0.05)])

# ==============================
# TRAIN & EVALUATE (10 epochs như gốc)
# ==============================
X_train, X_test, y_train, y_test = train_test_split(X_2d, y, test_size=0.2,
                                                    random_state=42, stratify=y)

class_weights = compute_class_weight('balanced', classes=np.arange(11), y=y_train)
class_weight_dict = dict(enumerate(class_weights))

def train_and_eval(build_fn, name):
    tf.keras.backend.clear_session()
    model = keras.Sequential([aug, build_fn()])
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    model.fit(X_train, y_train,
              epochs=10,
              batch_size=32,
              validation_split=0.1,
              class_weight=class_weight_dict,
              verbose=1,
              callbacks=[keras.callbacks.EarlyStopping(patience=10,
                                                       restore_best_weights=True)])
    pred = np.argmax(model.predict(X_test, verbose=0), axis=1)
    return {
        'accuracy' : accuracy_score(y_test, pred),
        'precision': precision_score(y_test, pred, average='macro', zero_division=0),
        'recall'   : recall_score(y_test, pred, average='macro', zero_division=0),
        'f1'       : f1_score(y_test, pred, average='macro', zero_division=0)
    }

print("\n=== Bắt đầu huấn luyện (10 epochs, KHÔNG trừ nền) ===")
res_dn = train_and_eval(build_densenet_2d, "DenseNet2D")
res_rn = train_and_eval(build_resnet_2d,   "ResNet2D")

# ==============================
# IN KẾT QUẢ CUỐI CÙNG
# ==============================
print("\n" + "="*75)
print("KẾT QUẢ KHI KHÔNG TRỪ NỀN (10 epochs)")
print("="*75)
print(f"DenseNet 2D (GADF)  →  Acc: {res_dn['accuracy']:.4f}  |  Prec: {res_dn['precision']:.4f}  |  Rec: {res_dn['recall']:.4f}  |  F1: {res_dn['f1']:.4f}")
print(f"ResNet   2D (GADF)  →  Acc: {res_rn['accuracy']:.4f}  |  Prec: {res_rn['precision']:.4f}  |  Rec: {res_rn['recall']:.4f}  |  F1: {res_rn['f1']:.4f}")
print("="*75)

# lưu metrics
import json
with open(f'{exp_dir}/results/metrics_NO_baseline.json', 'w') as f:
    json.dump({'DenseNet2D': res_dn, 'ResNet2D': res_rn}, f, indent=4)

Đang tạo 11.000 phổ GADF (không trừ nền)...
   → Ethanol: 400/999
   → Ethanol: 800/999
   → Methanol: 400/999
   → Methanol: 800/999
   → EM1_a: 400/999
   → EM1_a: 800/999
   → EM2_a: 400/999
   → EM2_a: 800/999
   → EM3_a: 400/999
   → EM3_a: 800/999
   → EM4_a: 400/999
   → EM4_a: 800/999
   → EM5_a: 400/999
   → EM5_a: 800/999
   → EM6_a: 400/999
   → EM6_a: 800/999
   → EM7_a: 400/999
   → EM7_a: 800/999
   → EM8_a: 400/999
   → EM8_a: 800/999
   → EM9_a: 400/999
   → EM9_a: 800/999

Hoàn tất! X_2d shape: (11000, 64, 64, 1)  |  Label distribution:
[1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000]

=== Bắt đầu huấn luyện (10 epochs, KHÔNG trừ nền) ===

Epoch 1/10
248/248 ━━━━━━━━━━━━━━━━━━━━ 129s 486ms/step - accuracy: 0.2310 - loss: 2.0297 - val_accuracy: 0.0966 - val_loss: 3.2822
Epoch 2/10
248/248 ━━━━━━━━━━━━━━━━━━━━ 97s 391ms/step - accuracy: 0.6500 - loss: 0.8104 - val_accuracy: 0.5273 - val_loss: 1.1841
Epoch 3/10
248/248 ━━━━━━━━━━━━━━━━━━━━ 97s 390ms/step - accurac